In [1]:
!git clone https://www.github.com/muxspace/facial_expressions.git

Cloning into 'facial_expressions'...
remote: Enumerating objects: 14214, done.
remote: Total 14214 (delta 0), reused 0 (delta 0), pack-reused 14214
Receiving objects: 100% (14214/14214), 239.65 MiB | 27.21 MiB/s, done.
Resolving deltas: 100% (223/223), done.
Checking out files: 100% (13996/13996), done.


In [2]:
import csv 
data={}
with open('/content/facial_expressions/data/legend.csv') as f:
  reader=csv.reader(f)
  next(reader)
  for row in reader:
    key=row[2].lower()
    if key in data:
      data[key].append(row[1])
    else:
      data[key]=[row[1]]


In [3]:
emotion_list=list(data.keys())

In [4]:
import os
os.mkdir('master_data')
os.mkdir('master_data/training')
os.mkdir('master_data/testing')

In [5]:
for emotion in emotion_list:
  os.mkdir(os.path.join('master_data/training/',emotion))
  os.mkdir(os.path.join('master_data/testing/',emotion))

In [6]:
from shutil import copyfile
split_size=0.8
for emotion,images in data.items():
  train_size=int(split_size*len(images))
  train_images=images[:train_size]
  test_images=images[train_size:]
  for image in train_images:
    source=os.path.join('/content/facial_expressions/images',image)
    dest=os.path.join('/content/master_data/training',emotion,image)
    copyfile(source,dest)
  for image in test_images:
    source=os.path.join('/content/facial_expressions/images',image)
    dest=os.path.join('/content/master_data/testing',emotion,image)
    copyfile(source,dest)




In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten

In [8]:
model=tf.keras.models.Sequential([
                                  Conv2D(16,(3,3,),activation='relu',input_shape=(100,100,3)),
                                  MaxPooling2D(2,2),
                                  Conv2D(32,(3,3),activation='relu'),
                                  MaxPooling2D(2,2),
                                  Conv2D(64,(3,3),activation='relu'),
                                  MaxPooling2D(2,2),
                                  Flatten(),
                                  Dense(512,activation='relu'),
                                  Dense(8,activation='softmax')

])
model.compile(optimizer=Adam(lr=0.01),loss='categorical_crossentropy',metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 16)        448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6400)              0

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [10]:
train_dir='/content/master_data/training'
test_dir='/content/master_data/testing'
train_datagen=ImageDataGenerator(rescale=1/255)
train_generator=train_datagen.flow_from_directory(train_dir,target_size=(100,100),class_mode='categorical',batch_size=128)
test_datagen=ImageDataGenerator(rescale=1/255)
test_generator=test_datagen.flow_from_directory(test_dir,target_size=(100,100),class_mode='categorical',batch_size=128)

Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [11]:
es=EarlyStopping(monitor='val_acc',patience=2,min_delta=0.01)



In [12]:
model.fit_generator(train_generator,epochs=10,verbose=1,validation_data=test_generator,callbacks=[es])

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
86/86 [==============================] - 48s 220ms/step - loss: 1.6241 - acc: 0.4828 - val_loss: 1.0401 - val_acc: 0.4267
Epoch 2/10
86/86 [==============================] - 18s 212ms/step - loss: 1.0151 - acc: 0.5241 - val_loss: 0.9929 - val_acc: 0.5525
Epoch 3/10
86/86 [==============================] - 18s 212ms/step - loss: 0.8334 - acc: 0.6938 - val_loss: 0.9082 - val_acc: 0.6218
Epoch 4/10
86/86 [==============================] - 19s 218ms/step - loss: 0.6982 - acc: 0.7524 - val_loss: 0.9361 - val_acc: 0.6645
Epoch 5/10
86/86 [==============================] - 19s 221ms/step - loss: 0.6351 - acc: 0.7783 - val_loss: 1.0615 - val_acc: 0.6568
Epoch 6/10
86/86 [==============================] - 19s 220ms/step - loss: 0.6067 - acc: 0.7839 - val_loss: 1.0897 - val_acc: 0.6565
